In [71]:
import urllib.request,urllib.parse,urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import lxml.html as lh
import pandas as pd

In [72]:
# ctx = ssl.create_default_context()
# ctx.check_hostname = False
# ctx.verify_mode = ssl.CERT_NONE

In [73]:
#html = urllib.request.urlopen(url,context = ctx).read()
#soup = BeautifulSoup(html,'lxml')

#Retrieve all anchor tags
#tags = soup.find('tr')
#for tag in tags:
#  print(tag.get('//tr',None))

In [74]:
#soup.get_text()

In [75]:
#tags

In [76]:
url = input('Enter = ')
response = requests.get(url)

Enter = https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [77]:
response

<Response [200]>

In [78]:
doc = lh.fromstring(response.content)
trs = doc.xpath('//tr') #All rows of a table

In [79]:
li=[]
i=0

for t in trs[0]: #accessing row_n = 1(header)
    i+=1
    name=t.text_content()
    print ('%d:%s'%(i,name))
    li.append((name,[]))

1:Postal Code

2:Community

3:Neighbourhood



In [80]:
trs[1] #this row has 3 elements which can be accessed using .iterchildren()

<Element tr at 0x7f37d40db868>

In [81]:
li

[('Postal Code\n', []), ('Community\n', []), ('Neighbourhood\n', [])]

In [82]:
for j in trs[1].iterchildren():
    print(j.text_content())

M1A

Not assigned

Not assigned



In [83]:
len(trs)

185

> * trs contains data about each row
* Lenght of the table is 180 + 1 but len(trs){[0:181]} is returning 185 because the page has some other rows in the bottom also which is added in trs as well. 
* In the below code cell, we're using a trick
> <pre>if len(T)!=3:
      break</pre>
It is because except the main table rows, other rows contains more data than 3 text items. When the rows of main table will be finished, this code iterates out of this loop. 
* **li** is the list of tuples where each tuple has an empty list
> <pre>i = 0
for t in T.iterchildren():
      data=t.text_content() 
      li[i][1].append(data)
      i += 1</pre>
* Above code is used to append in the list <pre>[('Postal Code\n', [here]), ('Community\n', [here]), ('Neighbourhood\n', [here])]</pre> at an index $(0,1,2)$ for each value of trs. 

In [84]:
for j in range(1,len(trs)):
    T=trs[j]
    if len(T)!=3:
        break
    
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        li[i][1].append(data)
        i+=1

li now is
<pre>[('Postal Code\n',['M1A','M2A'...]),('Community\n',['NA','North York'..]),...]

In [85]:
d={key:value for (key,value) in li}
df=pd.DataFrame(d)

In [86]:
df.head()

,Postal Code\n,Community\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [87]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postal Code\n,Community\n,Neighbourhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [88]:
df.columns = ['Postal code','Borough','Neighborhood']
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [89]:
df['Postal code'] = df['Postal code'].str.strip()
df['Borough'] = df['Borough'].str.strip()
df['Neighborhood'] = df['Neighborhood'].str.strip()

In [90]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [91]:
df.shape

(104, 3)

In [92]:
df.tail()

,Postal code,Borough,Neighborhood
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
103,,Canadian postal codes,


In [93]:
df.drop(df.index[df.Neighborhood ==''],inplace = True)

In [94]:
df.tail()

,Postal code,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [95]:
df.shape

(103, 3)

In [96]:
df.to_csv('2_toronto_locations_names.csv')